In [ ]:
# install packages
%cd /content
!git clone https://github.com/danhtran2mind/MotionDirector
%cd MotionDirector
!pip install -r requirements.txt -q

/content
Cloning into 'MotionDirector'...
remote: Enumerating objects: 657, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 657 (delta 108), reused 88 (delta 81), pack-reused 494 (from 1)
Receiving objects: 100% (657/657), 132.29 MiB | 50.34 MiB/s, done.
Resolving deltas: 100% (349/349), done.
/content/MotionDirector
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━

In [2]:
!pip install -q bitsandbytes unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.8 MB/s eta 0:00:00


In [3]:
import os
import shutil
import random

def copy_file_pairs(source_dir, dest_dir, max_pairs=20, seed=None):
    if seed is not None:
        random.seed(seed)
    os.makedirs(dest_dir, exist_ok=True)
    mp4_files = [f for f in os.listdir(source_dir) if f.endswith('.mp4')]
    selected_mp4_files = random.sample(mp4_files, min(len(mp4_files), max_pairs))
    for mp4 in selected_mp4_files:
        base = os.path.splitext(mp4)[0]
        txt = f"{base}.txt"
        if os.path.exists(os.path.join(source_dir, txt)):
            shutil.copy2(os.path.join(source_dir, mp4), os.path.join(dest_dir, mp4))
            shutil.copy2(os.path.join(source_dir, txt), os.path.join(dest_dir, txt))
    return len(selected_mp4_files)


In [4]:
import os

from huggingface_hub import snapshot_download

# Create directory if it doesn't exist
os.makedirs("data/ghibli/raw", exist_ok=True)

# Download the dataset using snapshot_download
snapshot_download(repo_id="raymondt/ghibi_t2v", 
                 local_dir="data/ghibli/raw", 
                 repo_type="dataset")

# Assuming the zip file is downloaded, unzip it to the target directory
import zipfile
zip_path = "data/ghibli/raw/studio_ghibli_wan14b_t2v_v01_dataset.zip"
extract_path = "data/ghibli/raw"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)



Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

studio_ghibli_wan14b_t2v_v01_dataset.zip:   0%|          | 0.00/300M [00:00<?, ?B/s]

In [5]:
# Copy the videos directory to the desired location
source = "data/ghibli/raw/videos/1920x1040"
dest = "data/ghibli/videos"

copied = copy_file_pairs(source, dest, max_pairs=240, seed=42)
print(f"Copied {copied} pairs to {dest}")

Copied 240 pairs to data/ghibli/videos


In [6]:
from huggingface_hub import snapshot_download
# Download ZeroScope model snapshot
repo_id = "cerspense/zeroscope_v2_576w"
snapshot_download(repo_id=repo_id,
                  local_dir="./ckpts/zeroscope_v2_576w")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

merges.txt: 0.00B [00:00, ?B/s]

model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

text2video_pytorch_model.pth:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

'/content/MotionDirector/ckpts/zeroscope_v2_576w'

In [9]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
# device

In [ ]:
%%writefile configs/config_multi_videos.yaml
# Pretrained diffusers model path.
pretrained_model_path: "./ckpts/zeroscope_v2_576w"
# pretrained_model_path: "./ckpts/text-to-video-ms-1.7b"
# The folder where your training outputs will be placed.
output_dir: "./zeroscope_v2_576w-Ghibli-LoRA"
# resume_step: 500
# resume_from_checkpoint: "./zeroscope_v2_576w-Scenery_Anime_Bright-lora/train_2025-07-10T13-46-57"
# lora_path: "zeroscope_v2_576w-Scenery_Anime_Bright-lora/checkpoint-500" # This argument is used for training resumption
# lora_path: zeroscope_v2_576w-Ghibli-LoRA/train_2025-07-13T06-46-47/checkpoint-200

dataset_types:
  - 'folder'

# Caches the latents (Frames-Image -> VAE -> Latent) to a HDD or SDD.
# The latents will be saved under your training folder, and loaded automatically for training.
# This both saves memory and speeds up training and takes very little disk space.
cache_latents: True


# If you have cached latents set to `True` and have a directory of cached latents,
# you can skip the caching process and load previously saved ones.
cached_latent_dir: null #/path/to/cached_latents
# cached_latent_dir: zeroscope_v2_576w-Ghibli-LoRA/train_2025-07-13T06-46-47/cached_latents

# Use LoRA for the UNET model.
use_unet_lora: True

# LoRA Dropout. This parameter adds the probability of randomly zeros out elements. Helps prevent overfitting.
# See: https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
lora_unet_dropout: 0.1

# Choose whether or not ito save the full pretrained model weights for both checkpoints and after training.
# The only time you want this off is if you're doing full LoRA training.
save_pretrained_model: True
# save_pretrained_model: True

# The rank for LoRA training. With ModelScope, the maximum should be 1024.
# VRAM increases with higher rank, lower when decreased.
lora_rank: 16

# Training data parameters
train_data:
  # 'multiple videos'
  path: "./data/ghibli/videos"
  # The width and height in which you want your training data to be resized to.
  width: 384
  height: 384

  # This will find the closest aspect ratio to your input width and height.
  # For example, 512x512 width and height with a video of resolution 1280x720 will be resized to 512x256
  use_bucketing: True
  gradient_accumulation_steps: 2
  batch_size: 1
  # The start frame index where your videos should start (Leave this at one for json and folder based training).
  sample_start_idx: 1

  # Used for 'folder'. The rate at which your frames are sampled. Does nothing for 'json' and 'single_video' dataset.
  fps: 16

  # For 'single_video' and 'json'. The number of frames to "step" (1,2,3,4) (frame_step=2) -> (1,3,5,7, ...).
  frame_step: 1

  # The number of frames to sample. The higher this number, the higher the VRAM (acts similar to batch size).
  n_sample_frames: 24

  # The prompt when using a a single video file
  # fallback_prompt: "A person is riding a bicycle."

# Validation data parameters.
validation_data:
  # A custom prompt that is different from your training dataset.
  prompt:
  - "Studio Ghibli style. The video showcases a vibrant and lively scene set in the early."
  - "Studio Ghibli style. A woman with black hair is holding a gun in her hand."

  # Whether or not to sample preview during training (Requires more VRAM).
  # sample_preview: True
  sample_preview: False

  # The number of frames to sample during validation.
  num_frames: 24

  # Height and width of validation sample.
  width: 384
  height: 384

  # Number of inference steps when generating the video.
  num_inference_steps: 15

  # CFG scale
  guidance_scale: 12

  # scale of spatial LoRAs, default is 0
  spatial_scale: 0

  # scale of noise prior, i.e. the scale of inversion noises
  noise_prior: 0

use_offset_noise: False
offset_noise_strength: 0.

# Learning rate for AdamW
learning_rate: 5e-4

# Weight decay. Higher = more regularization. Lower = closer to dataset.
adam_weight_decay: 1e-4

# Maximum number of train steps. Model is saved after training.
max_train_steps: 5000

# Saves a model every nth step.
checkpointing_steps: 5000

# How many steps to do for validation if sample_preview is enabled.
validation_steps: 5000

# Whether or not we want to use mixed precision with accelerate
mixed_precision: "fp16"
# mixed_precision: "no"

# Trades VRAM usage for speed. You lose roughly 20% of training speed, but save a lot of VRAM.
# If you need to save more VRAM, it can also be enabled for the text encoder, but reduces speed x2.
gradient_checkpointing: True
text_encoder_gradient_checkpointing: True

# Xformers must be installed for best memory savings and performance (< Pytorch 2.0)
enable_xformers_memory_efficient_attention: True
use_8bit_adam: True

# Use scaled dot product attention (Only available with >= Torch 2.0)
enable_torch_2_attn: True

Overwriting configs/config_multi_videos.yaml


In [8]:
# Train
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
!python main_train.py --config ./configs/config_multi_videos.yaml

2025-07-16 05:36:13.391674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752644173.574411     316 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752644173.625685     316 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Initializing the conversion map
{'rescale_betas_zero_snr', 'timestep_spacing'} was not found in config. Values will be initialized to default values.
An error occurred while trying to fetch ./ckpts/zeroscope_v2_576w: Error no file named diffusion_pytorch_model.safetensors found in directory ./ckpts/zeroscope_v2_576w.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
{'latents_mean', 'use_post_qu